In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

*Understanding the transaction data*

In [ ]:
#read in transaction data
trxDF = pd.read_parquet('data_science_take_home_transactions.parquet')
trxDF.dtypes

In [ ]:
#rename f0_ to amount
trxDF = trxDF.rename(columns={'f0_': 'amount'})
# use pandas type for date column
trxDF['date'] = pd.to_datetime(trxDF['date'])
trxDF.head()

In [ ]:
#Question to understand schema - Do any user accounts use multiple bank accounts?
count = trxDF.groupby('account_id')['bank_account_id'].nunique()
count[count>1].any()

*Visualize the payment data by user*

In [ ]:
import matplotlib.pyplot as plt

def visualize_user_payments(user_id, df):
    payments = df[(df['account_id'] == user_id) & (df['amount'] > 0)]

    plt.scatter(pd.to_datetime(payments['date']), payments['amount'])

    plt.xlabel('Date')
    plt.ylabel('Amount')
    plt.title('Payments Received by User ' + user_id)

    plt.show()


In [ ]:
import matplotlib.pyplot as plt

def visualize_user_payments_histogram(user_id, df):
    payments = df[(df['account_id'] == user_id) & (df['amount'] > 0)]

    plt.hist(payments['amount'], bins=3)

    plt.xlabel('Amount')
    plt.ylabel('Frequency')
    plt.title('Payments Histogram for User ' + user_id)

    plt.show()


In [ ]:
#visualize_user_payments(trxDF['account_id'].unique()[4],trxDF)
visualize_user_payments('h5aza5vad3b8p', trxDF)

In [ ]:
#visualize_user_payments_histogram(trxDF['account_id'].unique()[1],trxDF)
visualize_user_payments_histogram('h5aza5vad3b8p', trxDF)

In [ ]:
df_given_user = trxDF[(trxDF['account_id'] == 'zs569bfbe1pma') & (trxDF['amount'] > 750)]
df_given_user.sort_values(by=['amount'], inplace=True,ascending=False)
df_given_user

## Feature Extraction
### Extract features from the data for clustering
1. Use histogram data with three bins each as features - visualized above
2. Frequency metric
3. Percentage made up by top 2 payments monthly
4. Percentage made up by single largest payment monthly - could indicate pension or monthly income
5. Payment "regularity" metric - standard deviation of days between positive transaction
6. Number of unique payment names


In [ ]:
import datetime

# just last six months of data to start out with a cleaner dataset
today = datetime.date.today()
six_months_ago = today - pd.DateOffset(months=6)
trxDF_six_months = trxDF.loc[trxDF['date'] >= six_months_ago]

**histogram data into 3 bins**

In [ ]:
import numpy as np
# Get the payment amounts per user
payment_amounts = trxDF[trxDF['amount']>0].groupby('account_id')


In [ ]:
# Calculate the histogram for each user
histograms = payment_amounts.apply(lambda x: np.histogram(x['amount'], bins=3)[0])

histograms


**Get some generally applicable features based on payment frequency**


In [ ]:
payments = trxDF[trxDF['amount'] > 0].groupby('account_id')
date_range = pd.Timestamp.now() - payments['date'].min()
paymentsPerDay = payments.size() / date_range.apply(lambda x: x.days)
paymentsPerDay.sort_index(inplace=True, ascending=False)
paymentsPerDay.head()


**Metric to answer - how much of the total inbound value / month is coming from the largest payments**

In [ ]:
# How much income on average comes from top two payments per month?

# calculate the monthly income for each user
trxDF['month'] = trxDF['date'].dt.month

monthly_income = trxDF[trxDF['amount']>0].groupby(['account_id', 'month'])['amount'].sum().reset_index()

monthly_income.rename(columns={'amount': 'income'}, inplace=True)

In [ ]:

trxDF = trxDF.sort_values(by=['account_id', 'month', 'amount'], ascending=False)

# select the top 2 payments to each user and each month (could indicate biweekly payments)
top_2 = trxDF.groupby(['account_id', 'month']).head(2).reset_index(drop=True)

# find percentage of monthly income from the top 2 transactions for each user and each month
top_2 = top_2.merge(monthly_income, on=['account_id', 'month'])
top_2['percentage'] = top_2['amount'].div(top_2['income'])

avg_top2_pct = top_2.groupby('account_id')['percentage'].mean()
avg_top2_pct.sort_index(inplace=True, ascending=False)
avg_top2_pct.head()

In [ ]:
# select the largest payment to each user and each month (could indicate biweekly payments)
top_payment = trxDF.groupby(['account_id', 'month']).head(1).reset_index(drop=True)

# find percentage of monthly income from the top 2 transactions for each user and each month
top_payment = top_payment.merge(monthly_income, on=['account_id', 'month'])
top_payment['percentage'] = top_payment['amount'].div(top_payment['income'])

avg_top_pmnt_pct = top_payment.groupby('account_id')['percentage'].mean()#.reset_index()
avg_top_pmnt_pct.sort_index(inplace=True, ascending=False)
avg_top_pmnt_pct.head()

**Regularity of payments based on standard deviation**

In [ ]:
trxDF = trxDF.sort_values('date')

# std deviations of days between payment dates
def std_diff(group):
    return group['date'].diff().dt.days.std()

# group the dataframe by the account id and calculate std of time days payment dates
std_days_between = trxDF.groupby('account_id').apply(std_diff)
std_days_between.sort_index(inplace=True, ascending=False)
std_days_between.head()

**Number of unique positive transaction (payment) names**

In [ ]:
unique_payers = trxDF[trxDF['amount'] > 0].groupby('account_id')['name'].nunique()
unique_payers.sort_index(inplace=True, ascending=False)
unique_payers

# KMeans clustering to segment into three groups of users
We want to find three segments, namely gig workers, salaried (biweekly paydays), and pensioners (monthly paydays).

In [ ]:
avg_top2_pct.shape

In [ ]:
features = pd.concat([paymentsPerDay, std_days_between, unique_payers, avg_top_pmnt_pct,avg_top2_pct], axis=1,keys=['paymentsPerDay', 'std_days_between', 'unique_payers', 'avg_top_pmnt_pct','avg_top2_pct'])
features = features.dropna()
features.head()

In [ ]:
from sklearn.cluster import KMeans


In [ ]:
normalizedFeatures = (features - features.min()) / (features.max() - features.min())
normalizedFeatures.head()

In [ ]:

# Create and fit kmeans model
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(normalizedFeatures)

# Assign cluster labels to data
features["cluster"] = kmeans.labels_

In [ ]:
features[features['cluster'] == 0]

# Evaluation of results so far
Cluster 0 likely indicates salaried workers and some pension workers with regular payments that also make up a large percentage of their income. Cluster 1 seems to contain salaried workers as well with gig income also indicated by irregularity of payment timing and frequency of payments. Cluster 2 likely includes some salaried workers but with numbers of payments and unique payment names.

This model doesn't seem very accurate yet as we were primarily searching for segments with the three different types of income payment-frequencies. Cluster 1 and cluster 2 don't seem to be clearly delineated. This model also doesn't account for users with different types of incomes

**Improving the models**
- Binary classification models for each type of income.
- - This would enable features specified for each type of decision. Salaried workers will likely have paychecks with a certain minimum amount. It would make sense to filter for payments greater than $800 to search for biweekly paychecks
- - This would also account for multiple income types with each classification happening separately


**Use case for NLP?**
Naive approach: we could simply search for words / terms in the name of the transaction that would indicate what type of payment it is - income, loan, p2p, etc.
Even better: A standard LLM could likely be fine-tuned to classify each transaction line, considering all fields, to decide what type of payment it is.

Once the classification is made between income or otherwise, the type of income can be calculated fairly easily based on payment frequency